In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import shutil
import glob

In [10]:
ROOT_DIR = "/content/drive/MyDrive/recombined_2"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

In [11]:
number_of_images.items()

dict_items([('Malignant', 470), ('Benign', 300)])

In [12]:
def dataFolder(p,split):
  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

    for dir in os.listdir(ROOT_DIR):
        os.makedirs("./"+p+"/"+dir)

        for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)),
                                    size=(math.floor(split*number_of_images[dir])-10),
                                    replace=False):
            O = os.path.join(ROOT_DIR, dir, img)  #path
            D = os.path.join("./" +p, dir)
            shutil.copy(O, D)
            os.remove(O)

        else:
          print(f"{p} the folder exist")

In [13]:
dataFolder("train",0.8)

train the folder exist
train the folder exist


In [14]:
dataFolder("val",0.1)

val the folder exist
val the folder exist


In [15]:
dataFolder("test",0.1)

test the folder exist
test the folder exist


# Model Build

In [20]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

In [22]:
!pip install opencv-python

In [23]:
import cv2

In [24]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    fill_mode='nearest'
)
image_width = 224
image_height = 224
classes = ['Benign', 'Malignant']
# Resimleri yüklemek için flow_from_directory kullanın
train_generator = datagen.flow_from_directory(
    directory="/content/train",
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Her sınıf için resimleri ayrı ayrı arttırın
for cls in classes:
    # Sınıfın resimlerini yüklemek için flow_from_directory kullanın
    cls_generator = datagen.flow_from_directory(
        directory="/content/train",
        target_size=(image_width, image_height),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        classes=[cls]
    )

    # Resimleri arttırın ve kaydedin
    i = 0
    for batch in cls_generator:
        images, labels = batch
        for image in images:
            # Resmi kaydetmek için imwrite kullanın
            cv2.imwrite(os.path.join("/content/train", cls, f"augmented_{i}.jpg"), image * 255)
            i += 1
            if i >= 3000:  # Maksimum resim sayısına ulaşıldığında döngüyü durdur
                break
        else:
            continue
        break


Found 596 images belonging to 2 classes.
Found 230 images belonging to 1 classes.
Found 366 images belonging to 1 classes.


In [25]:
augmented_images = os.listdir("/content/train/Malignant")

# Count the number of augmented images
num_malignant_images = len(augmented_images)

# Print the number of malignant images
print("Number of malignant images:", num_malignant_images)

Number of malignant images: 3366


In [26]:
augmented_images = os.listdir("/content/train/Benign")

# Count the number of augmented images
num_benign_images = len(augmented_images)

# Print the number of benign images
print("Number of benign images:", num_benign_images)

Number of benign images: 3230


In [ ]:
def preprocessingImages1(path):
  """
  input : Path
  output : Preprocessed images
  """
  image_data = ImageDataGenerator

In [28]:
train_dir = "/content/train"

In [33]:
from keras.applications.mobilenet import preprocess_input
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(rescale=1/255., zoom_range=0.2,shear_range = 0.2, preprocessing_function = preprocess_input)
train_data_cancer = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="binary")

Found 6596 images belonging to 2 classes.


In [34]:
test_dir = "/content/test"
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
test_datagen = ImageDataGenerator(rescale=1/255., zoom_range=0.2,shear_range = 0.2, preprocessing_function = preprocess_input)
test_data_cancer = test_datagen.flow_from_directory(test_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="binary")

Found 57 images belonging to 2 classes.


In [35]:
val_dir = "/content/val"
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
val_datagen = ImageDataGenerator(rescale=1/255., zoom_range=0.2,shear_range = 0.2, preprocessing_function = preprocess_input)
val_data_cancer = val_datagen.flow_from_directory(val_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="binary")

Found 57 images belonging to 2 classes.


In [37]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.optimizers import Adam

In [38]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 1s 0us/step


In [39]:
for layer in base_model.layers:
    layer.trainable = False

In [40]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [41]:
X = Flatten()(base_model.output)
X = Dense(units = 1, activation='sigmoid')(X)

model = Model(base_model.input, X)


In [42]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

In [43]:
model.compile(optimizer = 'rmsprop', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [44]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
nc = ModelCheckpoint(filepath= "best_model.h5",monitor="val_accuracy", verbose = 1, save_best_only=True)

es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience = 3, verbose = 1)

cb = [nc, es]


In [46]:
model.fit_generator(train_data_cancer,
                    epochs = 40,
                    steps_per_epoch= 100,
                    validation_data = val_data_cancer,
                    validation_steps = 16,
                    callbacks = cb)

Epoch 1/40
100/100 [==============================] - ETA: 0s - loss: 1.0988 - accuracy: 0.5082

100/100 [==============================] - 36s 356ms/step - loss: 1.0988 - accuracy: 0.5082
Epoch 2/40
100/100 [==============================] - ETA: 0s - loss: 1.0971 - accuracy: 0.5163

100/100 [==============================] - 37s 367ms/step - loss: 1.0971 - accuracy: 0.5163
Epoch 3/40
100/100 [==============================] - ETA: 0s - loss: 1.1264 - accuracy: 0.5016

100/100 [==============================] - 37s 367ms/step - loss: 1.1264 - accuracy: 0.5016
Epoch 4/40
100/100 [==============================] - ETA: 0s - loss: 1.0683 - accuracy: 0.5125

100/100 [==============================] - 37s 365ms/step - loss: 1.0683 - accuracy: 0.5125
Epoch 5/40
100/100 [==============================] - ETA: 0s - loss: 1.1104 - accuracy: 0.5075

...

100/100 [==============================] - 37s 365ms/step - loss: 1.0161 - accuracy: 0.5434
Epoch 37/40
100/100 [==============================] - ETA: 0s - loss: 1.0602 - accuracy: 0.5369

100/100 [==============================] - 36s 361ms/step - loss: 1.0602 - accuracy: 0.5369
Epoch 38/40
100/100 [==============================] - ETA: 0s - loss: 1.0443 - accuracy: 0.5312

100/100 [==============================] - 37s 368ms/step - loss: 1.0443 - accuracy: 0.5312
Epoch 39/40
100/100 [==============================] - ETA: 0s - loss: 1.0281 - accuracy: 0.5296

100/100 [==============================] - 35s 351ms/step - loss: 1.0281 - accuracy: 0.5296
Epoch 40/40
100/100 [==============================] - ETA: 0s - loss: 1.0606 - accuracy: 0.5347

100/100 [==============================] - 37s 374ms/step - loss: 1.0606 - accuracy: 0.5347


In [48]:
acc = model.evaluate_generator(test_data_cancer)[1]
print(f"Test accuracy: {acc}")

Test accuracy: 0.4035087823867798
